# Wokflow  **TEST** con Full Bayesiana

## Inicializacion

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 05:43:45 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658595,35.2,1454652,77.7,1228568,65.7
Vcells,1222346,9.4,8388608,64.0,1975158,15.1


In [ ]:
plocal <- list()

plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 200L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.4


plocal$APO <- 1
plocal$ksemillerio <- 200


In [ ]:
PARAM <- list()
PARAM$experimento <- "zAC03-entrega03VF-80a"
PARAM$semilla_primigenia <- 170029

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset de c02
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz")

# leo el dataset de c03
dataset09 <- fread("~/datasets/competencia_03_crudo.csv.gz")

# uno los dataset
dataset <- rbind(dataset, dataset09, fill = TRUE)
rm(dataset09)
gc()

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-03 05:43:45 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,759087,40.6,1454652,77.7,1454652,77.7
Vcells,555646591,4239.3,1481537416,11303.3,1296635121,9892.6


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,767968,41.1,1454652,77.7,1454652,77.7
Vcells,560571904,4276.9,1481537416,11303.3,1296635121,9892.6


[1] "2025-12-03 05:44:05 UTC"

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

### Data Quality

#### Variables rotas > NA

In [ ]:
# 1. Identificar las columnas numéricas a chequear
numeric_types <- c("integer", "numeric", "double", "integer64")
all_cols <- names(dataset)

# Obtenemos los nombres de las columnas que son numéricas
cols_a_chequear <- all_cols[sapply(dataset, function(x) class(x)[1] %in% numeric_types)]

cols_a_chequear <- setdiff(cols_a_chequear, c("numero_de_cliente", "foto_mes"))

# 2. Calcular el porcentaje de ceros por mes
resumen_wide <- dataset[,
                        lapply(.SD, function(x) sum(x == 0, na.rm = TRUE) / .N),
                        by = foto_mes,
                        .SDcols = cols_a_chequear]

# 3. Convertir (melt) la tabla a formato largo
resumen_long <- melt(resumen_wide,
                     id.vars = "foto_mes",
                     variable.name = "variable",
                     value.name = "porcentaje_ceros")

# 4. Filtrar y crear la lista de problemas
lista_problemas <- resumen_long[porcentaje_ceros == 1,
                                .(foto_mes, variable)]

setorder(lista_problemas, foto_mes, variable)

# 5. Mostrar el resultado
print("Columnas que son 100% CERO en un mes:")
print(lista_problemas)

[1] "Columnas que son 100% CERO en un mes:"
    foto_mes                           variable
       <int>                             <fctr>
 1:   201904  ctarjeta_visa_debitos_automaticos
 2:   201904 mttarjeta_visa_debitos_automaticos
 3:   201905                      mrentabilidad
 4:   201905               mrentabilidad_annual
 5:   201905                        mcomisiones
 6:   201905                    mactivos_margen
 7:   201905                    mpasivos_margen
 8:   201905                  ccomisiones_otras
 9:   201905                  mcomisiones_otras
10:   201910                      mrentabilidad
11:   201910               mrentabilidad_annual
12:   201910                        mcomisiones
13:   201910                    mactivos_margen
14:   201910                    mpasivos_margen
15:   201910        ccajeros_propios_descuentos
16:   201910        mcajeros_propios_descuentos
17:   201910           ctarjeta_visa_descuentos
18:   201910           mtarjeta_visa_descuen

In [ ]:
cat("\nIniciando transformación a NA...\n")

for(i in 1:nrow(lista_problemas)) {

  # Obtenemos el mes y el nombre de la variable
  f <- lista_problemas$foto_mes[i]
  v <- as.character(lista_problemas$variable[i]) # Nombre de la columna como string

  # Determinamos el tipo de NA (Integer o Double)
  na_value <- if (class(dataset[[v]])[1] == "integer") {
      NA_integer_
    } else {
      NA_real_
    }

  dataset[foto_mes == f, (v) := na_value]

}

cat("¡Transformación completa!\n")



Iniciando transformación a NA...
¡Transformación completa!


### Data Drifting

#### Aguinaldo

In [ ]:
# --- Parámetros ---
vars_a_suavizar <- c("mpayroll",
                     "mpayroll2",
                     "mcaja_ahorro",
                     "mcuenta_corriente",
                     "mtarjeta_visa_consumo",
                     "mtarjeta_master_consumo",
                     "mautoservicio",
                     "mcuentas_saldo",
                     "minversion1_pesos",
                     "mplazo_fijo_pesos",
                     "mplazo_fijo_dolares"
)

umbral_aumento <- 1.5 # 50% de aumento para activar

# Asegurar el orden
setorder(dataset, numero_de_cliente, foto_mes)

# Crear columna temporal de mes calendario
dataset[, kmes_temp := foto_mes %% 100]

for (v in vars_a_suavizar) {
  cat("Suavizando (solo meses 6, 7  y 12):", v, "\n")

  # 1. Calcular Lag y Lead para toda la columna
  col_lag <- paste0(v, "_lag_tmp")
  col_lead <- paste0(v, "_lead_tmp")

  dataset[, (col_lag) := shift(get(v), 1, type="lag"), by = numero_de_cliente]
  dataset[, (col_lead) := shift(get(v), 1, type="lead"), by = numero_de_cliente]

  condicion_suavizado <-
    dataset$kmes_temp %in% c(6, 7, 12) &
    !is.na(dataset[[col_lag]]) &
    !is.na(dataset[[col_lead]]) &
    dataset[[v]] >= (umbral_aumento * dataset[[col_lag]])

  #    Contar casos afectados
  cant_afectados <- sum(condicion_suavizado, na.rm = TRUE)

  if (cant_afectados > 0) {
    cat("  -> Casos corregidos:", cant_afectados, "\n")

    #    Reemplazar por el promedio (lag + lead) / 2
    dataset[condicion_suavizado,
            (v) := (get(col_lag) + get(col_lead)) / 2]
  } else {
    cat("  -> Ningún caso cumplió el criterio.\n")
  }

  # 3. Limpiar columnas temporales
  dataset[, (col_lag) := NULL]
  dataset[, (col_lead) := NULL]
}

# Limpiar columna auxiliar de mes
dataset[, kmes_temp := NULL]

cat("Proceso de suavizado completado.\n")
gc()

Suavizando (solo meses 6, 7  y 12): mpayroll 
  -> Casos corregidos: 799698 
Suavizando (solo meses 6, 7  y 12): mpayroll2 
  -> Casos corregidos: 1173604 
Suavizando (solo meses 6, 7  y 12): mcaja_ahorro 
  -> Casos corregidos: 437901 
Suavizando (solo meses 6, 7  y 12): mcuenta_corriente 
  -> Casos corregidos: 908205 
Suavizando (solo meses 6, 7  y 12): mtarjeta_visa_consumo 
  -> Casos corregidos: 275481 
Suavizando (solo meses 6, 7  y 12): mtarjeta_master_consumo 
  -> Casos corregidos: 613445 
Suavizando (solo meses 6, 7  y 12): mautoservicio 
  -> Casos corregidos: 490767 
Suavizando (solo meses 6, 7  y 12): mcuentas_saldo 
  -> Casos corregidos: 346960 
Suavizando (solo meses 6, 7  y 12): minversion1_pesos 
  -> Casos corregidos: 1140977 
Suavizando (solo meses 6, 7  y 12): mplazo_fijo_pesos 
  -> Casos corregidos: 1170143 
Suavizando (solo meses 6, 7  y 12): mplazo_fijo_dolares 
  -> Casos corregidos: 1051869 
Proceso de suavizado completado.


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,783668,41.9,1454652,77.7,1454652,77.7
Vcells,563065529,4295.9,1481537416,11303.3,1296635121,9892.6


#### Cargamos los índices

In [ ]:
require(data.table)

indices_externos <- data.table(
  foto_mes = c(201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109),
  TC_BLUE = c("37,5", "39", "43,65", "46", "46", "43,8", "45,2", "63", "61,25", "69", "69,25", "78,5", "78", "78,5", "83,5", "118", "125", "126", "136", "135", "146", "169", "155", "166", "153", "146", "141", "150", "157", "168", "180,5", "182", "186"),
  TC_OFICIAL = c("38,28", "40,14", "44,4", "45,36", "46,1", "43,7", "45,02", "62,04", "59,82", "63,23", "62,93", "62,99", "63,03", "64,26", "66,58", "69,16", "70,76", "74,07", "76,39", "78,36", "80,63", "83,89", "86,72", "89,87", "92,7", "95,12", "97,69", "98,9", "100,09", "101,17", "102,06", "103,14", "104,30"),
  TCRM = c("116,5734916", "116,5625812", "119,8157416", "120,2150629", "119,4975567", "114,8101224", "109,8715912", "127,7565287", "130,6961531", "129,7797576", "126,9467229", "123,5951113", "120,8372653", "118,9226283", "114,5040182", "111,5867304", "111,7626924", "116,6307121", "117,9781145", "118,9579865", "119,6485855", "118,1736932", "119,6199112", "123,235859", "122,8366223", "122,3839332", "119,0439564", "116,7635839", "117,2617246", "116,9865438", "113,7702507", "111,9430182", "107,0965229"),
  UVA = c("32,03", "32,86", "33,97", "35,42", "36,89", "38,03", "39,2", "40,16", "41,26", "43,43", "45,42", "47,16", "49,05", "50,49", "51,62", "52,95", "54,24", "55,06", "56,09", "57,17", "58,52", "60,16", "61,94", "64,32", "66,54", "69,04", "71,92", "74,87", "78,07", "81,13", "83,82", "86,42", "88,76"),
  IPC = c("189,6101", "196,7501", "205,9571", "213,0517", "219,5691", "225,537", "230,494", "239,6077", "253,7102", "262,0661", "273,2158", "283,4442", "289,8299", "295,666", "305,5515", "310,1243", "314,9087", "321,9738", "328,2014", "337,0632", "346,6207", "359,657", "371,0211", "385,8826", "401,5071", "415,8595", "435,8657", "453,6503", "468,725", "483,6049", "498,0987", "510,3942", "528,4968"),
  IPIM = c("279", "288,4", "300,2", "314", "329,5", "334,9", "335,2", "372,8", "388,3", "402,3", "424", "439,7", "446,3", "451,3", "455,6", "449,7", "451,3", "467,8", "484,4", "504,2", "522,9", "547,3", "570,1", "595,2", "628,3", "666,5", "692,4", "725,5", "748,8", "772,3", "789,5", "809,4", "831,9"),
  ripte_indice = c("4042", "4198,76", "4444,6", "4533,03", "4676,25", "4753,19", "4948,27", "5039,93", "5199,08", "5467,59", "5554,15", "5666,48", "6066,07", "6445,13", "6500,72", "6510,18", "6521,87", "6670,93", "6908,52", "6945,86", "7076,47", "7401,81", "7495,03", "7643,41", "7784,1", "8263,33", "8665,19", "9201,59", "9311,61", "9660,13", "10089,96", "10326,11", "10762,48"),
  tasa_i = c("3,71", "3,09", "3,81", "4,45", "4,42", "3,96", "4,14", "4,84", "4,91", "4,06", "3,68", "3,29", "2,83", "2,65", "2,30", "1,54", "2,21", "2,47", "2,43", "2,47", "2,47", "2,65", "2,81", "2,85", "2,85", "2,80", "2,84", "2,84", "2,84", "2,84", "2,84", "2,85", "2,85"),
  tasa_r = c("0,79", "-0,68", "-0,85", "1,01", "1,28", "1,23", "1,89", "0,81", "-0,94", "0,73", "-0,59", "-0,40", "0,52", "0,64", "-0,97", "0,04", "0,70", "0,27", "0,52", "-0,23", "-0,32", "-1,11", "-0,38", "-1,10", "-1,11", "-0,77", "-1,87", "-1,21", "-0,44", "-0,35", "-0,15", "0,34", "-0,67")
)

# Trabajamos el formato
indices_externos[, foto_mes := as.integer(foto_mes)]

# Reemplazar la coma por el punto en todas las columnas excepto 'foto_mes'
columnas_numericas <- names(indices_externos)[-1]

indices_externos[, (columnas_numericas) := lapply(.SD, function(x) as.numeric(gsub(",", ".", x))),
                 .SDcols = columnas_numericas]

# convertir a tasa
cols_tasas <- c("tasa_i", "tasa_r")
indices_externos[, (cols_tasas) := lapply(.SD, function(x) x / 100), .SDcols = cols_tasas]

# Verificar la estructura de la tabla
str(indices_externos)

Classes ‘data.table’ and 'data.frame':	33 obs. of  10 variables:
 $ foto_mes    : int  201901 201902 201903 201904 201905 201906 201907 201908 201909 201910 ...
 $ TC_BLUE     : num  37.5 39 43.6 46 46 ...
 $ TC_OFICIAL  : num  38.3 40.1 44.4 45.4 46.1 ...
 $ TCRM        : num  117 117 120 120 119 ...
 $ UVA         : num  32 32.9 34 35.4 36.9 ...
 $ IPC         : num  190 197 206 213 220 ...
 $ IPIM        : num  279 288 300 314 330 ...
 $ ripte_indice: num  4042 4199 4445 4533 4676 ...
 $ tasa_i      : num  0.0371 0.0309 0.0381 0.0445 0.0442 0.0396 0.0414 0.0484 0.0491 0.0406 ...
 $ tasa_r      : num  0.0079 -0.0068 -0.0085 0.0101 0.0128 0.0123 0.0189 0.0081 -0.0094 0.0073 ...
 - attr(*, ".internal.selfref")=<externalptr> 


#### Rankings

In [ ]:
rank_normalizar_por_mes_monetarias <- function(p_dataset) {

  # 1. Define el vector con los nombres BASE de las columnas monetarias
  columnas_monetarias_base <- c(
    "mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen",
    "mpasivos_margen", "mcuenta_corriente_adicional", "mcuenta_corriente",
    "mcaja_ahorro", "mcaja_ahorro_adicional", "mcaja_ahorro_dolares",
    "mcuentas_saldo", "mautoservicio", "mtarjeta_visa_consumo",
    "mtarjeta_master_consumo", "mprestamos_personales", "mprestamos_prendarios",
    "mprestamos_hipotecarios", "mplazo_fijo_dolares", "mplazo_fijo_pesos",
    "minversion1_pesos", "minversion1_dolares", "minversion2", "mpayroll",
    "mpayroll2", "mcuenta_debitos_automaticos", "mtarjeta_visa_debitos_automaticos",
    "mttarjeta_master_debitos_automaticos", "mpagodeservicios", "mpagomiscuentas",
    "mcajeros_propios_descuentos", "mtarjeta_visa_descuentos",
    "mtarjeta_master_descuentos", "mcomisiones_mantenimiento", "mcomisiones_otras",
    "mforex_buy", "mforex_sell", "mtransferencias_recibidas", "mtransferencias_emitidas",
    "mextraccion_autoservicio", "mcheques_depositados", "mcheques_emitidos",
    "mcheques_depositados_rechazados", "mcheques_emitidos_rechazados", "matm",
    "matm_other", "Master_mfinanciacion_limite", "Master_msaldototal",
    "Master_msaldopesos", "Master_msaldodolares", "Master_mconsumospesos",
    "Master_mconsumosdolares", "Master_mlimitecompra", "Master_madelantopesos",
    "Master_madelantodolares", "Master_mpagado", "Master_mpagospesos",
    "Master_mpagosdolares", "Master_mconsumototal", "Master_mpagominimo",
    "Visa_mfinanciacion_limite", "Visa_msaldototal", "Visa_msaldopesos",
    "Visa_msaldodolares", "Visa_mconsumospesos", "Visa_mconsumosdolares",
    "Visa_mlimitecompra", "Visa_madelantopesos", "Visa_madelantodolares",
    "Visa_mpagado", "Visa_mpagospesos", "Visa_mpagosdolares", "Visa_mconsumototal",
    "Visa_mpagominimo"
  )

  # 2. Selecciona solo las columnas que realmente existen en el dataset
  columnas_a_rankear <- intersect(columnas_monetarias_base, names(p_dataset))


  cat("Columnas a rankear por mes (sobrescribiendo):", length(columnas_a_rankear), "\n")
  if (length(columnas_a_rankear) == 0) {
    cat("Advertencia: Ninguna de las columnas monetarias especificadas se encontró en el dataset.\n")
    return(invisible(p_dataset))
  }

  for (col in columnas_a_rankear) {

    p_dataset[, (col) := {
      v <- .SD[[1]]
      r <- numeric(length(v))

      pos_idx <- which(v > 0)
      neg_idx <- which(v < 0)

      if (length(pos_idx) > 0) {
        r[pos_idx] <- frank(v[pos_idx], ties.method = "average") / length(pos_idx)
      }
      if (length(neg_idx) > 0) {
        r[neg_idx] <- -frank(-v[neg_idx], ties.method = "average") / length(neg_idx)
      }

      r[v == 0] <- 0

      # Segunda normalización para asegurar el rango [-1, 1]
      max_r_pos <- max(r[pos_idx], 0, na.rm = TRUE)
      min_r_neg <- min(r[neg_idx], 0, na.rm = TRUE)

      if (max_r_pos > 0) r[pos_idx] <- r[pos_idx] / max_r_pos
      if (min_r_neg < 0) r[neg_idx] <- r[neg_idx] / abs(min_r_neg)

      .(r)
    }, by = foto_mes, .SDcols = col]
  }



  cat("Rankeo completado. Las columnas fueron sobrescritas.\n")
  gc()
  invisible(p_dataset)
}

In [ ]:
# aplico la función
rank_normalizar_por_mes_monetarias(dataset)



Columnas a rankear por mes (sobrescribiendo): 72 
Rankeo completado. Las columnas fueron sobrescritas.


### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

#### Creación de kmes y ctrx quarter normalizado

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-12-03 05:47:19 UTC"

#### Cepo

In [ ]:
cat("Creando variable flag 'cepo_activo' (Desde Nov-2019)...\n")

# 1. Inicializar en 0
dataset[, cepo_activo := 0L]

# 2. Marcar con 1 desde Noviembre 2019 en adelante
dataset[foto_mes >= 201911, cepo_activo := 1L]

cat("Variable 'cepo_activo' creada. Distribución:\n")
print(dataset[, .N, by = cepo_activo][order(cepo_activo)])

Creando variable flag 'cepo_activo' (Desde Nov-2019)...
Variable 'cepo_activo' creada. Distribución:
   cepo_activo       N
         <int>   <int>
1:           0 1290293
2:           1 3592758


#### Stress financiero

In [ ]:
cat("Creando Ratios de Estrés Financiero...\n")

# 1. Ratio Deuda / Límite (Saturación de Tarjeta)
dataset[, limite_total := (rowSums(.SD, na.rm=T)), .SDcols = c("Master_mlimitecompra", "Visa_mlimitecompra")]
dataset[, saldo_total := (rowSums(.SD, na.rm=T)), .SDcols = c("Master_msaldototal", "Visa_msaldototal")]

dataset[, ratio_deuda_limite := 0.0]
dataset[limite_total > 0, ratio_deuda_limite := saldo_total / limite_total]

# 2. Ratio Consumo / Ingreso (Capacidad de Pago)
if (!"payroll_total" %in% names(dataset)) {
   dataset[, payroll_total := rowSums(.SD, na.rm=T), .SDcols = intersect(c("mpayroll", "mpayroll2"), names(dataset))]
}
dataset[, consumo_total_tc := (rowSums(.SD, na.rm=T)), .SDcols = c("mtarjeta_visa_consumo", "mtarjeta_master_consumo")]

dataset[, ratio_consumo_ingreso := 0.0]
dataset[payroll_total > 0, ratio_consumo_ingreso := consumo_total_tc / payroll_total]

# 3. Limpieza
dataset[, c("limite_total", "saldo_total", "consumo_total_tc") := NULL]

Creando Ratios de Estrés Financiero...


### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]


# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 3
dataset[,
  paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 6
dataset[,
  paste0(cols_lagueables, "_lag6") := shift(.SD, 6, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 12
dataset[,
  paste0(cols_lagueables, "_lag12") := shift(.SD, 12, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
  dataset[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  dataset[, paste0(vcol, "_delta6") := get(vcol) - get(paste0(vcol, "_lag6"))]
  dataset[, paste0(vcol, "_delta12") := get(vcol) - get(paste0(vcol, "_lag12"))]
}

Sys.time()

[1] "2025-12-03 05:50:57 UTC"

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- TRUE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [ ]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 2070

[1] "2025-12-03 05:52:04 UTC"

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202109)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202107
)

PARAM$train_final$undersampling <- 0.1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [ ]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

set.seed( PARAM$semilla_primigenia ) #CHECK

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-12-03 05:52:10 UTC"

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [ ]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 64L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale

  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...

  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-12-03 05:52:11 UTC"

In [ ]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [ ]:
# detach("package:lightgbm", unload= TRUE)

In [ ]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/cutoff/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm



In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 370837 725929 629891 522659 401993 895771 294023 552649 829727 377459
 [11] 526649 608207 628207 127261 938059 772159 988129 695297 917633 853703
 [21] 870407 673193 555661 534529 882289 914647 397181 641203 599999 414893
 [31] 980179 995989 324161 938617 596419 672041 462113 114041 202717 832159
 [41] 248827 826849 124717 306193 322633 514903 101483 212869 347539 377051
 [51] 925063 436013 645647 319591 874091 384691 668273 387173 159463 250673
 [61] 575689 579613 606443 331141 263951 583981 501593 716731 716279 928513
 [71] 359389 179593 915017 723587 260983 147089 447893 454451 862783 558721
 [81] 191801 409967 854569 376501 339589 245941 649423 658349 304021 872237
 [91] 637771 513479 668959 843211 193799 213097 223403 888983 334549 470513
[101] 300877 494443 221393 554959 907811 709531 573383 549707 741457 106433
[111] 229837 380881 661253 726419 314771 432281 191299 430883 109423 413087
[121] 444937 349903 112207 975151 285221 669839 949853 366193 222979 434087
[131] 144719 681089 818309 786829 429043 337489 398471 330623 391199 305237
[141] 616489 768437 594137 185711 292267 940801 697327 194239 749863 957403
[151] 309929 296279 386963 469379 561809 526679 730559 899161 576427 236503
[161] 285757 867509 618679 975287 961613 641633 837521 255191 352399 511891
[171] 671287 185057 133649 148579 347957 321571 518191 852913 145459 621541
[181] 206399 139721 505711 167267 281431 151343 885061 644129 950179 164429
[191] 564989 521429 435583 775963 252827 202973 252457 197969 393161 252017

In [ ]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [ ]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 473099 columnas 2074 


[1] "2025-12-03 05:52:20 UTC"

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-12-03 08:06:23 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
# dataset de future, AHORA 202109

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

In [ ]:

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [ ]:
# Aplicar el ksemillerio (Promediar probabilidades) ---
vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0

# (PARAM$train_final$semillas tiene 30 semillas en BO-001)
for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( file.exists( arch_modelo ) )
  {
    modelo_final <- lgb.load(arch_modelo) # leo del disco
    vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
    qacumulados <- qacumulados + 1
    rm(modelo_final)
    gc()
  } else {
    cat("Advertencia: No se encontró el modelo para la semilla", sem, "\n")
  }
}

if (qacumulados == 0) {
  stop("ERROR: No se encontró ningún modelo en la carpeta /modelitos/. No se puede predecir.")
}

vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
cat("Predicciones promediadas de", qacumulados, "modelos.\n")


#  Guardar la tabla de probabilidades ---
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]

# grabo las probabilidad del modelo
fwrite(
  tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

cat("Tabla de probabilidades guardada en 'prediccion.txt'.\n")
Sys.time()


Predicciones promediadas de 200 modelos.
Tabla de probabilidades guardada en 'prediccion.txt'.


[1] "2025-12-03 08:12:04 UTC"

In [ ]:

dir.create("kaggle", showWarnings = FALSE)

# 1. ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

# 2. Defino el número de corte
envios <- 11000

# 3. Establezco como "1" los que se envían (como en tu lógica original)
tb_prediccion[, Predicted := 0L]
tb_prediccion[1:envios, Predicted := 1L]

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# 4. Grabo el archivo
fwrite(
  # Filtro por Predicted == 1L y selecciono SOLO numero_de_cliente
  tb_prediccion[Predicted == 1L, list(numero_de_cliente)],
  file = archivo_kaggle,
  sep = ",",
  col.names = FALSE
)


cat("Submission (solo IDs de envío, sin encabezado) generada y guardada en:", archivo_kaggle, "\n")

Submission (solo IDs de envío, sin encabezado) generada y guardada en: ./kaggle/KAzAC03-entrega03VF-80a_11000.csv 


In [ ]:
Sys.time()

[1] "2025-12-03 08:12:05 UTC"